<a href="https://colab.research.google.com/github/sumitpatel0718/ShopSense/blob/main/major__project_matching_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Load the dataset from the Excel file
df = pd.read_excel("liver_matching_dataset.xlsx")

# Extract recipient details from the first row (assuming it is the recipient)
recipient = df.iloc[0]

# Use all remaining rows as donor candidates
donor_candidates = df.iloc[1:].copy()  # copy to avoid SettingWithCopyWarning

def compute_matching_score(recipient, donor):
    """
    Compute a matching score between a recipient and a donor candidate.
    Lower scores indicate better compatibility.
    """
    # 1. Donor BMI penalty: penalize if BMI exceeds an ideal threshold (e.g., 25)
    bmi_penalty = max(0, donor['donor_BMI'] - 25)

    # 2. Liver function penalty: normalized sum of liver enzyme values and bilirubin.
    #    Here we assume that higher values indicate poorer liver function.
    liver_function_score = (
        donor['donor_AST'] / 100 +
        donor['donor_ALT'] / 100 +
        donor['donor_bilirubin'] / 2 +
        donor['donor_ALP'] / 150 +
        donor['donor_GGT'] / 80
    )

    # 3. Steatosis penalty: higher fatty infiltration is worse.
    steatosis_penalty = donor['donor_steatosis'] / 50  # normalize (max assumed at 50%)

    # 4. Donor risk index penalty: normalize by its maximum expected value (assumed 5)
    risk_penalty = donor['donor_risk_index'] / 5

    # 5. Donor age penalty: younger donors are generally preferred (normalize to max 70 years)
    age_penalty = donor['donor_age'] / 70

    # 6. Graft-to-recipient size ratio penalty: deviation from ideal ratio of 1.0
    graft_size_penalty = abs(donor['graft_to_recipient_size_ratio'] - 1.0)

    # Weights for each component (tweak these based on clinical importance)
    w1, w2, w3, w4, w5, w6 = 2.0, 1.5, 1.0, 1.5, 1.0, 2.0

    score = (
        w1 * bmi_penalty +
        w2 * liver_function_score +
        w3 * steatosis_penalty +
        w4 * risk_penalty +
        w5 * age_penalty +
        w6 * graft_size_penalty
    )

    return score

# Compute matching score for each donor candidate relative to the recipient
donor_candidates['matching_score'] = donor_candidates.apply(
    lambda row: compute_matching_score(recipient, row), axis=1
)

# Sort the donor candidates by matching score (lowest is best)
matching_donors = donor_candidates.sort_values('matching_score')

# Display the top 5 matching donors with key donor information and matching score
top_matches = matching_donors[[
    'donor_age', 'donor_gender', 'donor_BMI', 'donor_AST', 'donor_ALT',
    'donor_bilirubin', 'donor_ALP', 'donor_GGT', 'donor_steatosis',
    'donor_risk_index', 'donor_liver_volume', 'graft_to_recipient_size_ratio',
    'matching_score'
]].head()

print("Top 5 matching donors for the recipient (first row):")
print(top_matches)


Top 5 matching donors for the recipient (first row):
     donor_age donor_gender  donor_BMI  donor_AST  donor_ALT  donor_bilirubin  \
923         22       Female       24.9       27.1       50.2             0.15   
9           40       Female       18.8       21.7       44.5             0.26   
886         18       Female       22.6       32.3       21.3             1.39   
52          31       Female       24.4       69.0       56.5             0.27   
349         20       Female       22.9       21.9       71.7             0.34   

     donor_ALP  donor_GGT  donor_steatosis  donor_risk_index  \
923       84.4       10.9             37.2              1.18   
9         42.3       20.1             45.6              1.96   
886      100.2       12.6              4.6              1.16   
52        31.1       17.0             17.8              1.14   
349       51.9       21.5             16.8              3.71   

     donor_liver_volume  graft_to_recipient_size_ratio  matching_score  
92